In [1]:
!python -V

Python 3.12.7


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

In [5]:
from sklearn.metrics import root_mean_squared_error

In [6]:
import mlflow

In [7]:
mlflow.set_tracking_uri('http://ec2-3-148-222-145.us-east-2.compute.amazonaws.com:5000')

In [8]:
mlflow.get_tracking_uri()

'http://ec2-3-148-222-145.us-east-2.compute.amazonaws.com:5000'

In [9]:
# Test Run
with mlflow.start_run():
    mlflow.log_param('test_param', 123)
    mlflow.log_metric('test_metric', 0.95)

🏃 View run funny-shrew-446 at: http://ec2-3-148-222-145.us-east-2.compute.amazonaws.com:5000/#/experiments/0/runs/9c42a1ff14c4483e9953c626167e40d5
🧪 View experiment at: http://ec2-3-148-222-145.us-east-2.compute.amazonaws.com:5000/#/experiments/0


In [10]:
mlflow.set_experiment('nyc_taxi_experiment')

<Experiment: artifact_location='s3://mlflow-artifacts-experiments/1', creation_time=1747163755802, experiment_id='1', last_update_time=1747163755802, lifecycle_stage='active', name='nyc_taxi_experiment', tags={}>

This function reads in the parquet files for the green taxi trip data. It calculates the duration of a trip in minutes and keeps trips that have a duration of 1 to 60 minutes. It also updates the categorical features of PULocationID and DOLocationID from numbers to string.

In [11]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [12]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [13]:
len(df_train), len(df_val)

(73908, 61921)

In [14]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [15]:
categorical = ['PU_DO'] # ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [16]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715206128955

In [18]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

Let's try a run with MLFlow training the Lasso linear model.

In [19]:
with mlflow.start_run():
    mlflow.set_tag('developer', 'michelle')

    mlflow.log_param('train-data-path', './data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', './data/green_tripdata_2021-02.parquet')

    alpha = 0.1
    mlflow.log_param('alpha', alpha)
    lor = Lasso(alpha)
    lor.fit(X_train, y_train)

    y_pred = lor.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)

    with open('models/lasso.bin', 'wb') as f_out:
        pickle.dump((dv, lor), f_out)
    mlflow.log_artifact(local_path='models/lasso.bin', artifact_path='models_pickle')

🏃 View run grandiose-owl-124 at: http://ec2-3-148-222-145.us-east-2.compute.amazonaws.com:5000/#/experiments/1/runs/e310989bab9146c293a1c92283224285
🧪 View experiment at: http://ec2-3-148-222-145.us-east-2.compute.amazonaws.com:5000/#/experiments/1


Now try experimenting with training an XGBoost model.

In [20]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

/home/ubuntu/anaconda3/lib/python3.12/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(


In [21]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [22]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('developer', 'michelle')
        mlflow.set_tag('model', 'xgboost2')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric('rmse', rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [23]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:squarederror',
    'seed': 42
}

In [ ]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.24936                                                                  
[1]	validation-rmse:7.84082                                                                  
[2]	validation-rmse:7.21262                                                                  
[3]	validation-rmse:6.93644                                                                  
[4]	validation-rmse:6.80900                                                                  
[5]	validation-rmse:6.73969                                                                  
[6]	validation-rmse:6.70423                                                                  
[7]	validation-rmse:6.67831                                                                  
[8]	validation-rmse:6.66354                                                                  
[9]	validation-rmse:6.64898                                                                  
[10]	validation-rmse:6.63850                                

In [ ]:
mlflow.end_run()

In [ ]:
print('XGBoost Hyperparameter Tuning Complete')